In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER_IN = 175
VER_OUT = 175
PART = 0

In [ ]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [ ]:
users = pd.read_parquet(f'../../data/infer_with_features/train_v{VER_IN}_LB_{PART}.pqt')

In [ ]:
print( users.shape )
users.head()

In [ ]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')
#df['type'] = df['type'].map({'clicks':0,'carts':1,'orders':2}).astype('int8')

print( df.shape )
df.head()

In [ ]:
df = df.sort_values(['session','ts']).reset_index(drop=True)

In [ ]:
tmp = df.groupby('session').aid.agg('last').rename('last')

In [ ]:
%%time
users = users.merge(tmp.to_pandas(), left_on='user', right_index=True, how='left')

In [ ]:
users.head()

In [ ]:
%%time
x = users.item.values
y = users['last'].values

In [ ]:
users = users.drop(['last'],axis=1)

In [ ]:
embed = np.load('../../data/item_user_features/item_embed_32_LB.npy')
z = np.sqrt( np.sum(embed * embed,axis=1) )
embed = embed/ np.reshape(z,(-1,1))
embed.shape
#(1855603, 32)

In [ ]:
%%time
e1 = embed[x,]
e2 = embed[y,]

In [ ]:
e1.shape

In [ ]:
%%time
z = np.sum( e1*e2,axis=1 )

In [ ]:
users['z1'] = z

In [ ]:
users.z1.dtype

In [ ]:
users.head()

In [ ]:
users2 = cudf.DataFrame( users[['user','item']] )
print( users2.shape )

In [ ]:
for k in range(5):
    tmp = df.loc[df['type']!=0].reset_index(drop=True) 
    tmp['n'] = tmp.groupby('session').aid.cumcount()
    tmp = tmp.loc[tmp.n==k]
    tmp = tmp[['session','aid']]
    tmp.columns = ['user','last_buy']

    users2 = users2.merge(tmp, on='user', how='left')

    x = users2.loc[users2.last_buy.notnull()].item.to_numpy()
    y = users2.loc[users2.last_buy.notnull()]['last_buy'].to_numpy().astype('int32')

    e1 = embed[x,]
    e2 = embed[y,]

    z = np.sum( e1*e2,axis=1 )

    users2[f'z{k+2}'] = -1
    users2.loc[users2.last_buy.notnull(),f'z{k+2}'] = z

    users2 = users2.drop(['last_buy'],axis=1)
    
    print(k,users2.shape)

In [ ]:
del df, tmp, e1, e2, z, x, y, embed
gc.collect()

In [ ]:
print( users2.shape )
users2.head()

In [ ]:
print( users.shape)
users.head()

In [ ]:
%%time
users = users.merge(users2.drop_duplicates(['user','item']).to_pandas(), on=['user','item'], how='left')

In [ ]:
print( users.shape )
users.head()

In [ ]:
for c in ['z1']+[f'z{x+2}' for x in range(5)]:
    print(c)
    users[c] = users[c].astype('float32')

# Write Parquet

In [ ]:
for c in users.columns:
    print(c, users[c].dtype)
    if str(users[c].dtype) in ['Float32','Int32','float64','int64']:
        print('###',c)
        users[c] = users[c].astype('float32')

In [ ]:
%%time
users.to_parquet(f'../../data/infer_with_features/train_v{VER_OUT}_LB_{PART}.pqt',index=False)